# Data centric
CHEMIN Bastien  
GUITON Clément

In [23]:
import pandas as pd

## Importation des données

In [24]:
df_customers = pd.read_csv('datas/olist_customers_dataset.csv')
df_geolocation = pd.read_csv('datas/olist_geolocation_dataset.csv')
df_order_items = pd.read_csv('datas/olist_order_items_dataset.csv')
df_order_payments = pd.read_csv('datas/olist_order_payments_dataset.csv')
df_order_reviews = pd.read_csv('datas/olist_order_reviews_dataset.csv')
df_orders = pd.read_csv('datas/olist_orders_dataset.csv')
df_products = pd.read_csv('datas/olist_products_dataset.csv')
df_sellers = pd.read_csv('datas/olist_sellers_dataset.csv')
df_category_name_translation = pd.read_csv('datas/product_category_name_translation.csv')

## Extraction des données

### Structure des données


#### Customer
- **Customer ID:** [customer_id]
- **Customer Unique ID:** [customer_unique_id]
- **Zip Code Prefix:** [customer_zip_code_prefix]
- **City:** [customer_city]
- **State:** [customer_state]

##### Geolocation
- **Zip Code Prefix:** [geolocation_zip_code_prefix]
- **Latitude:** [geolocation_lat]
- **Longitude:** [geolocation_lng]
- **City:** [geolocation_city]
- **State:** [geolocation_state]

#### Order Items
- **Order ID:** [order_id]
- **Order Item ID:** [order_item_id]
- **Product ID:** [product_id]
- **Seller ID:** [seller_id]
- **Shipping Limit Date:** [shipping_limit_date]
- **Price:** [price]
- **Freight Value:** [freight_value]

#### Order Payment
- **Order ID:** [order_id]
- **Payment Sequential:** [payment_sequential]
- **Payment Type:** [payment_type]
- **Payment Installments:** [payment_installments]
- **Payment Value:** [payment_value]

#### Order Reviews
- **Review ID:** [review_id]
- **Order ID:** [order_id]
- **Review Score:** [review_score]
- **Review Comment Title:** [review_comment_title]
- **Review Comment Message:** [review_comment_message]
- **Review Creation Date:** [review_creation_date]
- **Review Answer Timestamp:** [review_answer_timestamp]

#### Order
- **Order ID:** [order_id]
- **Customer ID:** [customer_id]
- **Order Status:** [order_status]
- **Order Purchase Timestamp:** [order_purchase_timestamp]
- **Order Approved At:** [order_approved_at]
- **Order Delivered Carrier Date:** [order_delivered_carrier_date]
- **Order Delivered Customer Date:** [order_delivered_customer_date]
- **Order Estimated Delivery Date:** [order_estimated_delivery_date]

#### Products
- **Product ID:** [product_id]
- **Product Category Name:** [product_category_name]
- **Product Name Length:** [product_name_length]
- **Product Description Length:** [product_description_length]
- **Product Photos Qty:** [product_photos_qty]
- **Product Weight (g):** [product_weight_g]
- **Product Length (cm):** [product_length_cm]
- **Product Height (cm):** [product_height_cm]
- **Product Width (cm):** [product_width_cm]

#### Sellers
- **Seller ID:** [seller_id]
- **Seller Zip Code Prefix:** [seller_zip_code_prefix]
- **Seller City:** [seller_city]
- **Seller State:** [seller_state]

